<a href="https://colab.research.google.com/github/toriving/TIL/blob/master/pytorch/datasets/processing_data_in_a_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets

     |████████████████████████████████| 2.0MB 20.5MB/s 
     |████████████████████████████████| 194kB 47.9MB/s 
     |████████████████████████████████| 890kB 42.0MB/s 
     |████████████████████████████████| 3.2MB 49.8MB/s 
     |████████████████████████████████| 112kB 55.8MB/s 
     |████████████████████████████████| 245kB 53.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=2e3b5cd124aea3c4b8fe312fdf1f12eed6064e3dd6a935205681582ba3f141c4
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from datasets import load_dataset
dataset = load_dataset('glue', 'mrpc', split='train')

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4. Subsequent calls will reuse this data.


In [3]:
# datasets.Datasets.sort() and datasets.Dataset.map() 은 캐싱되어 다시 사용됨

In [4]:
# datasets.Dataset.cleanup_cache_files() 으로 삭제 가능

In [5]:
dataset['label'][:10]

[1, 0, 1, 0, 1, 1, 0, 1, 0, 0]

In [6]:
sorted_dataset = dataset.sort('label')

In [7]:
sorted_dataset['label'][:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [8]:
sorted_dataset['label'][-10:]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [9]:
shuffled_dataset = sorted_dataset.shuffle(seed=42)

In [10]:
shuffled_dataset['label'][:10]

[1, 1, 1, 0, 1, 1, 1, 1, 1, 0]

In [11]:
small_dataset = dataset.select([0, 10, 20, 30, 40, 50])

In [12]:
len(small_dataset)

6

In [16]:
start_with_ar = dataset.filter(lambda example: example['sentence1'].startswith('Ar'))

In [17]:
start_with_ar['sentence1']

['Around 0335 GMT , Tab shares were up 19 cents , or 4.4 % , at A $ 4.56 , having earlier set a record high of A $ 4.57 .',
 'Arison said Mann may have been one of the pioneers of the world music movement and he had a deep love of Brazilian music .',
 'Arts helped coach the youth on an eighth-grade football team at Lombardi Middle School in Green Bay .',
 'Around 9 : 00 a.m. EDT ( 1300 GMT ) , the euro was at $ 1.1566 against the dollar , up 0.07 percent on the day .',
 "Arguing that the case was an isolated example , Canada has threatened a trade backlash if Tokyo 's ban is not justified on scientific grounds .",
 'Artists are worried the plan would harm those who need help most - performers who have a difficult time lining up shows .']

In [18]:
even_dataset = dataset.filter(lambda example, indice: indice % 2 == 0, with_indices=True)

In [19]:
len(even_dataset)

1834

In [20]:
len(dataset) / 2

1834.0

In [21]:
dataset.train_test_split(test_size=0.1)

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3301
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 367
    })
})

In [22]:
0.1 * len(dataset)

366.8

In [28]:
dataset.shard(5, 4)

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 733
})

In [29]:
dataset = dataset.rename_column("sentence1", "sentenceA")
dataset = dataset.rename_column("sentence2", "sentenceB")

In [30]:
dataset

Dataset({
    features: ['sentenceA', 'sentenceB', 'label', 'idx'],
    num_rows: 3668
})

In [31]:
dataset = dataset.remove_columns("label")

In [32]:
dataset

Dataset({
    features: ['sentenceA', 'sentenceB', 'idx'],
    num_rows: 3668
})

In [34]:
dataset = dataset.remove_columns(['sentenceA', 'sentenceB'])

In [35]:
dataset

Dataset({
    features: ['idx'],
    num_rows: 3668
})

In [37]:
dataset = load_dataset('glue', 'mrpc', split='train')

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)


In [38]:
dataset.features

{'idx': Value(dtype='int32', id=None),
 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], names_file=None, id=None),
 'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None)}

In [39]:
from datasets import ClassLabel, Value

In [40]:
new_features = dataset.features.copy()

In [41]:
new_features

{'idx': Value(dtype='int32', id=None),
 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], names_file=None, id=None),
 'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None)}

In [42]:
new_features["label"] = ClassLabel(names=['negative', 'positve'])

In [43]:
new_features['idx'] = Value('int64')

In [44]:
dataset = dataset.cast(new_features)

In [45]:
dataset.features

{'idx': Value(dtype='int64', id=None),
 'label': ClassLabel(num_classes=2, names=['negative', 'positve'], names_file=None, id=None),
 'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None)}

In [46]:
squad = load_dataset("squad", split="train")

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/0fd9e01360d229a22adfe0ab7e2dd2adc6e2b3d6d3db03636a51235947d4c6e9. Subsequent calls will reuse this data.


In [47]:
squad

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

In [48]:
flattened_squad = squad.flatten()

In [49]:
flattened_squad

Dataset({
    features: ['answers.answer_start', 'answers.text', 'context', 'id', 'question', 'title'],
    num_rows: 87599
})

In [51]:
squad["answers"][0]

{'answer_start': [515], 'text': ['Saint Bernadette Soubirous']}

In [52]:
small_dataset = dataset.select(range(10))

In [53]:
small_dataset

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 10
})

In [56]:
small_dataset.map(lambda example: print(len(example['sentence1'])))

103


103
89
105
119
105
97
88
54
85
108



Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 10
})

In [57]:
def add_prefix(example):
    example['sentence1'] = 'My sentence: ' + example['sentence1']
    return example
updated_dataset = small_dataset.map(add_prefix)
updated_dataset['sentence1'][:5]

['My sentence: Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 "My sentence: Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .",
 'My sentence: They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added .',
 'My sentence: Around 0335 GMT , Tab shares were up 19 cents , or 4.4 % , at A $ 4.56 , having earlier set a record high of A $ 4.57 .',
 'My sentence: The stock rose $ 2.11 , or about 11 percent , to close Friday at $ 21.51 on the New York Stock Exchange .']

In [58]:
updated_dataset = small_dataset.map(lambda example: {'sentence1': 'My sentence: ' + example['sentence1']})
updated_dataset['sentence1'][:5]

['My sentence: Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 "My sentence: Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .",
 'My sentence: They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added .',
 'My sentence: Around 0335 GMT , Tab shares were up 19 cents , or 4.4 % , at A $ 4.56 , having earlier set a record high of A $ 4.57 .',
 'My sentence: The stock rose $ 2.11 , or about 11 percent , to close Friday at $ 21.51 on the New York Stock Exchange .']

In [59]:
updated_dataset = small_dataset.map(lambda example: {'new_sentence': example['sentence1']}, remove_columns=['sentence1'])
updated_dataset.column_names

['idx', 'label', 'new_sentence', 'sentence2']

In [60]:
updated_dataset = small_dataset.map(lambda example, idx: {'sentence2': f'{idx}: ' + example['sentence2']}, with_indices=True)
updated_dataset['sentence2'][:5]

['0: Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 "1: Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 .",
 "2: On June 10 , the ship 's owners had published an advertisement on the Internet , offering the explosives for sale .",
 '3: Tab shares jumped 20 cents , or 4.6 % , to set a record closing high at A $ 4.57 .',
 '4: PG & E Corp. shares jumped $ 1.63 or 8 percent to $ 21.03 on the New York Stock Exchange on Friday .']

In [61]:
def chunk_examples(examples):
    chunks = []
    for sentence in examples['sentence1']:
        chunks += [sentence[i:i + 50] for i in range(0, len(sentence), 50)]
    return {'chunks': chunks}
chunked_dataset = dataset.map(chunk_examples, batched=True, remove_columns=dataset.column_names)
chunked_dataset
chunked_dataset[:10]

{'chunks': ['Amrozi accused his brother , whom he called " the ',
  'witness " , of deliberately distorting his evidenc',
  'e .',
  "Yucaipa owned Dominick 's before selling the chain",
  ' to Safeway in 1998 for $ 2.5 billion .',
  'They had published an advertisement on the Interne',
  't on June 10 , offering the cargo for sale , he ad',
  'ded .',
  'Around 0335 GMT , Tab shares were up 19 cents , or',
  ' 4.4 % , at A $ 4.56 , having earlier set a record']}

In [ ]:
from datasets import concatenate_datasets, load_dataset
bookcorpus = load_dataset("bookcorpus", split="train")
wiki = load_dataset("wikipedia", "20200501.en", split="train")
wiki = wiki.remove_columns("title")  # only keep the text
assert bookcorpus.features.type == wiki.features.type
bert_dataset = concatenate_datasets([bookcorpus, wiki])

Dataset bookcorpus downloaded and prepared to /root/.cache/huggingface/datasets/bookcorpus/plain_text/1.0.0/af844be26c089fb64810e9f2cd841954fd8bd596d6ddd26326e4c70e2b8c96fc. Subsequent calls will reuse this data.


In [ ]:
bert_dataset